In [3]:
import requests
from collections import Counter
from web3 import Web3, HTTPProvider
connection = Web3(HTTPProvider('https://mainnet.infura.io/v3/6873c06628bc4c50af981c610f2f0fc1'))
current_block = connection.eth.blockNumber
print ("Latest Ethereum block number", current_block)

Latest Ethereum block number 15039874


In [4]:
number_of_blocks = 100 #ca 5400 blocks per day so 5400 * 30 162'000 blocks per month
starting_block = current_block - number_of_blocks

candidates_dict = {}

for i in range(number_of_blocks):
    print(i, " of ", number_of_blocks)
    block_number = starting_block - i
    block = connection.eth.get_block(block_number)
    hex_transaction_hashes = block["transactions"]

    for i in hex_transaction_hashes:
        transaction_hash = i.hex() 
        transaction = connection.eth.get_transaction(transaction_hash)
        value = transaction["value"] / (10**18)
        if (value > 0.2):
            receiver = transaction["to"] 
            if (receiver in candidates_dict):
                list = candidates_dict[receiver]
                list.append(value)
                candidates_dict[receiver] = list
            else:
                candidates_dict[receiver] = [value]
            

candidates_dict

0  of  100
1  of  100
2  of  100
3  of  100
4  of  100
5  of  100
6  of  100
7  of  100
8  of  100
9  of  100
10  of  100
11  of  100
12  of  100
13  of  100
14  of  100
15  of  100
16  of  100
17  of  100
18  of  100
19  of  100
20  of  100
21  of  100
22  of  100
23  of  100
24  of  100
25  of  100
26  of  100
27  of  100
28  of  100
29  of  100
30  of  100
31  of  100
32  of  100
33  of  100
34  of  100
35  of  100
36  of  100
37  of  100
38  of  100
39  of  100
40  of  100
41  of  100
42  of  100
43  of  100
44  of  100
45  of  100
46  of  100
47  of  100
48  of  100
49  of  100
50  of  100
51  of  100
52  of  100
53  of  100
54  of  100
55  of  100
56  of  100
57  of  100
58  of  100
59  of  100
60  of  100
61  of  100
62  of  100
63  of  100
64  of  100
65  of  100
66  of  100
67  of  100
68  of  100
69  of  100
70  of  100
71  of  100
72  of  100
73  of  100
74  of  100
75  of  100
76  of  100
77  of  100
78  of  100
79  of  100
80  of  100
81  of  100
82  of  100
83  of  100
84

{'0x881D40237659C251811CEC9c364ef91dC08D300C': [10.0,
  0.774,
  1.15,
  1.3,
  3.5,
  0.37703,
  0.28,
  0.7808,
  1.1,
  0.9,
  1.87,
  2.0,
  0.258,
  2.11,
  1.39,
  1.3,
  3.6,
  5.3,
  0.9,
  0.495,
  0.4,
  0.9],
 '0x3A7B75c381707D99534f7d039E2bFD7ffC3c5cfe': [734.2466410999999],
 '0xb04c0EB29C72cEBC467b9d4944D29116fa02C44a': [2.100236042266304,
  2.380311548157444,
  10.047379693007487,
  3.787889153643542],
 '0x6262998Ced04146fA42253a5C0AF90CA02dfd2A3': [4.2511248381447615,
  0.3222565943202551,
  0.30234447074575227,
  0.3169559078433502,
  0.528561259088021,
  0.28255162511335236,
  1.919502259088021,
  0.995020935496037,
  0.570285870012166,
  2.998992560012166,
  1.2867549250640842],
 '0x7758E507850dA48cd47df1fB5F875c23E3340c50': [1.2498797379155566,
  0.25070285701600736],
 '0x1aB118Fa08CA16C12C4458E12055aF91020C2675': [0.897],
 '0x20cCdeDB9814c83bA2D663fC04f88c7a61aA706d': [0.499297649125376],
 '0xD09860ab6326373e22579e878B7cdc7E36B96194': [0.35],
 '0x7a250d5630B4cF53973

In [5]:
new_candidates_dict = {}

# for (key, value) in candidates_dict.items():
#     if (any(x > 1 for x in value)):
#         new_candidates_dict[key] = value

new_candidates_dict = candidates_dict
len(new_candidates_dict)

1464

In [6]:
all_info_for_addresses = []
transactions_from_most_common_sender_for_addresses = []

most_common_sender_per_address = []

counter = 1464
updates = 0

for address in new_candidates_dict.keys():
    print(updates, " of ", counter)
    updates += 1
    try:
        url = f"https://api.covalenthq.com/v1/1/address/{address}/transactions_v2/?quote-currency=USD&page-size=150&key=ckey_06de8b133f2948eb88f41f10539"
        res = requests.get(url).json()

        transactions = res["data"]["items"]
        transactions_cleaned = []
        sender_addresses = []

        num_transaction_in_22 = len([1 for x in transactions if x["block_signed_at"][0:4] == "2022"])
        if (num_transaction_in_22 > 100):
            #print("skipping")
            continue

        for trans in transactions:
            if(trans["value_quote"] is not None and trans["value_quote"] > 100 and trans["from_address"] != address):
                sender_addresses.append(trans["from_address"])
                transactions_cleaned.append({"timestamp": trans["block_signed_at"], "from": trans["from_address"], "from_label": trans["from_address_label"], "value": float(trans["value"]) / (10**18), "valueUSD": trans["value_quote"]})

        all_info_for_addresses.append({"address": address, "data": transactions_cleaned})

        c = Counter(sender_addresses)
        most_common_address = c.most_common(1)[0][0]

        transactions_from_most_common_sender = []
        year_months = []
        for trans in transactions_cleaned:
            if (trans["from"] == most_common_address):
                transactions_from_most_common_sender.append(trans)
                year_months.append(trans["timestamp"][0:7])

        if (len(set(year_months)) >= 10):
            transactions_from_most_common_sender_for_addresses.append({"address": address, "data": transactions_from_most_common_sender})
            most_common_sender_per_address.append({"address": address, "payer_data": c.most_common(1)})

        #print(transactions_from_most_common_sender)
        print(most_common_sender_per_address)

        if (counter == 0):
            break
        else:
            print(counter)
            counter -= 1
    except:
        print("error came up - continuing")
        continue

0  of  1464
error came up - continuing
1  of  1464
[]
1464
2  of  1463
3  of  1463
4  of  1463
5  of  1463
[]
1463
6  of  1462
[]
1462
7  of  1461
[]
1461
8  of  1460
error came up - continuing
9  of  1460
[]
1460
10  of  1459
[]
1459
11  of  1458
error came up - continuing
12  of  1458
13  of  1458
14  of  1458
error came up - continuing
15  of  1458
[]
1458
16  of  1457
[]
1457
17  of  1456
[]
1456
18  of  1455
[]
1455
19  of  1454
[]
1454
20  of  1453
error came up - continuing
21  of  1453
22  of  1453
23  of  1453
24  of  1453
25  of  1453
[]
1453
26  of  1452
[]
1452
27  of  1451
[]
1451
28  of  1450
29  of  1450
[]
1450
30  of  1449
31  of  1449
[]
1449
32  of  1448
[]
1448
33  of  1447
34  of  1447
[]
1447
35  of  1446
[]
1446
36  of  1445
[]
1445
37  of  1444
38  of  1444
[]
1444
39  of  1443
40  of  1443
[]
1443
41  of  1442
error came up - continuing
42  of  1442
[]
1442
43  of  1441
[]
1441
44  of  1440
[]
1440
45  of  1439
46  of  1439
[]
1439
47  of  1438
[]
1438
48  of  

In [1]:
transactions_from_most_common_sender_for_addresses

NameError: name 'transactions_from_most_common_sender_for_addresses' is not defined